## Setup

Connect the Co-lab with Drive for persistence storage of co-lab notebooks

In [1]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive/')


Mounted at /content/drive/


Install the requirements file

In [2]:
!pip install -qqq -r "/content/drive/My Drive/Colab Notebooks/crewNewsRequirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -qqq crewai crewai_tools langchain_google_genai langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.6 MB/s eta 0:00:00


Load the API Key and libraries

In [4]:
from crewai import Agent
from langchain_google_genai import ChatGoogleGenerativeAI

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [5]:
## load the api key
import os
from google.colab import userdata
google_api_key = userdata.get('google-api-key')
os.environ["SERPER_API_KEY"] = userdata.get('serper-key')
os.environ["GROQ_API_KEY"] = userdata.get('groq-key')
os.environ["OPENAI_API_KEY"] = userdata.get('openai-key')

## Define Tools
Use SerperDevTool for google search

In [6]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# initialize the SerperDevTool with the API Key for Google search capabilities
search_tool = SerperDevTool(
    n_results = 10,
)
scrape_tool = ScrapeWebsiteTool()

Creating custom tool for image generation

1. Started with a DiffusionPipeline model, a image generation model, from hugging face, which wasn't able to create images consistently
2. Later switched to Dall-e-2 which produces great images. The cost/image isn't that bad:

  1024×1024   is   $0.020 / image

  512×512     is   $0.018 / image

  256×256     is   $0.016 / image
2. This tool allows to embed an image in the blog to make it visually appealing

Enter the Blog Topic here:

In [7]:
topic = "The impact of AI in modern Human Society"

In [8]:
!pip install -qqq openai

In [9]:
from openai import OpenAI
from crewai_tools import BaseTool
client = OpenAI()


class ImageTool(BaseTool):
    name: str = "image_tool"
    description: str = "Generates an image from a given prompt."

    def _run(self, text:str):
        response = client.images.generate(
          model="dall-e-2",
          prompt=topic,
          size="512x512",
          quality="standard",
          n=1,
        )

        image_url = response.data[0].url
        return image_url

# Create an instance of the ImageTool
image_tool = ImageTool()

## Create Agents


Creating 4 agents: researcher, content writer, image_generator and an editor

I played with a few LLM models, llama is performing great at the moment. It does get stuck in the loop sometimes depending the topic.

Haven't tried the gpt models yet since they are costly. Wanted to see how GPT-4o-mini would compare against llama, since it's cheaper, but read an article online suggesting it also gets stuck in the loop much more thatn GPT-4o. So, that will be something I will work on to find the most effective LLM model for this project.  

In [10]:
## Call the gemini model
llm = ChatGoogleGenerativeAI( model = "gemini-1.5-flash",
                             verbose = True,
                              temperature = 0.5,
                              google_api_key = google_api_key)
# llm2 = ChatOpenAI(model_name='gpt-4', temperature=0)
from langchain_groq import ChatGroq
llama_llm = ChatGroq(model="llama-3.1-70b-versatile", temperature = 0)

In [11]:
!pip install -qqq -U langchain-openai

In [12]:
from langchain_openai import ChatOpenAI

gpt_llm = ChatOpenAI(model_name='gpt-4o-2024-08-06', temperature=0)

Create a senior researcher agent with memory and verbose True mode

In [13]:
memory = True

In [25]:
researcher = Agent(
    role = "Senior Researcher",
    goal = f'Uncover ground breaking technologies in {topic}',
    verbose = True,
    memory = memory,
    backstory = """
    You are an expert in the field of research and have been tasked with finding the latest articles and news regarding the topic at hand.
    You have access to a wide range of resources and are able to quickly and efficiently find the information needed to complete the task.
    You should provide significant context for the next agent to use.
    DO NOT get stuck in the same internet article for too long!!
    """,
    tools = [search_tool, scrape_tool],
    llm = gpt_llm,
    allow_delegation = False
)

Create a writer agent with custom tools responsible for writing blog articles

In [26]:
blog_writer = Agent(
    role = "Content Writer",
    goal = f"Format the blog content and adhere to rules laid out by Benjamin Dreyer in the book Dreyer's English",
    verbose = True,
    memory = memory,
    backstory = ("You're a creative writer with a flair for storytelling and a background in journalism."
     "With a talent for turning complex information into engaging and readable content."
     "Back your every arguments with facts and figures. Your assest lies in"
      "creating compelling content based on comprehensive research."),
    tools = [],
    llm = gpt_llm,
    allow_delegation = False
)

Creating a prompt for text-image generation

In [27]:
image_generator = Agent(
    role = "Image Generator",
    goal = f'Generate an image on: {topic}',
    verbose = True,
    memory = memory,
    backstory = ("You're an experienced artist with amazing skills of turning text into an image."
        "You are good at telling a story through images that can accurately convey the"
        "message contained within the text."),
    tools = [image_tool],
    llm = gpt_llm,
    allow_delegation = False
)

The editor agent combines the blog-text and the image together

In [28]:
editor = Agent(
    role="Editor",
    goal="Write a compelling blog article along with an image",
    verbose=True,
    memory=memory,
    backstory = ("You're the chief editor of a New York Times."
        "You are good at polishing the blog created by writers."
        "Your final output is a compelling blog article along with an image"
        "generated by the image_generator agent. The image MUST be aligned to the center."),
    llm = gpt_llm,
    allow_delegation = False
)

## Define Tasks
Create three different tasks for each of the agents

In [18]:
from crewai import Task

# Research task
research_task = Task(
    agent = researcher,
    description = (f"Conduct comprehensive research on {topic} by leveraging"
                   "various online sources including news articles, blogs, research papers, and social media."
                  "Gather the most relevant, up-to-date, and accurate information to create a "
                  "detailed report that highlights key insights, trends, and data points."
                  "This report will serve as the foundational content for the writer agent "
                  "to create an engaging narrative. Focus on identifying pros and cons "
                  "creating a narrative with balanced views both positive and negative aspects."
                  "Your final report should clearly articulate the key points."
                  "DO NOT get stuck in the same internet article for too long!!"),
    expected_output = f"""A comprehensive 4 or 5 paragraph report on {topic} based on the research findings.
              Your report should contain balanced content on the public speculation, expert's views,
              and latest trends. Use data, facts, and figures to back any of your claims and points.

              """,
    tools = [search_tool, scrape_tool],
    human_input = True
)

# Writing task with language model configuration
write_task = Task(
    agent = blog_writer,
    description = ("Create a compelling and well-structured narrative based on the detailed report provided by"
                  "the researcher agent. Turn the complex information into an engaging and readable blog article,"
                   "ensuring that the content is both informative and captivating. "
                   "Accurately reflect the latest trends and insights on the given topic, "
                   "making it accessible and interesting for a broad audience."
                   "Also include the image generated by image_generator agent."),
    expected_output = f"A 4-5 paragraph blog article approx 500 words on {topic} formatted as markdown."+
          """
          The content is based on the research findings provided by the researcher agent.
          Use facts and figures to back any of your claims and points.
          The blog should look something like this in markdown format:
                          <Blog Title>
                          <Introduction and background>
                          <Main Points>
                          <Conclusion>
          """,
    tools = [],
    async_execution = False,
    output_file = "blog_article.md"
)

Creating tasks to generate images, very similar to the above two tasks

In [19]:
image_task = Task(
    agent = image_generator,
    description = ("Generate an image from the given prompt"),
    expected_output = "A visually appealing image, in jpeg format." +
          "The image must capture the main points of the blog"+
          "The image must be able to display in the Google Co-lab notebook",
    tools = [image_tool],
    async_execution = False,
    output_file = "blog_image.jpeg"
)

In [20]:
edit_task = Task(
    agent = editor,
    description = ("Write a compelling blog article along with an image"),
    expected_output = (f"A 4-5 paragraph blog article approximately 500 words on {topic} "
                       "formatted as markdown along with an image provided by image_task."+
          """
          Use the image generated by the image_generator agent in the blog.
          Main the same font style throught the blog.

          The image MUST BE strictly under the 'title' section aligned to the center.
          Include Subtitles for the main points and conclusion paragraphs.
          The final blog MUST Look something like this:
                          <Title>
                          <Image generated by the image_generator agent>
                          <Introduction>
                          <Main Points>
                          <Conclusion>
          """),
    tools = [],
    async_execution = False,
    output_file = "final_blog.md"
)

To Display images on the notebook

In [21]:
# Needed if you would like to display images in the notebook
!pip install --upgrade --quiet  opencv-python scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 40.0 MB/s eta 0:00:00


## Define Crew
Assign tasks to each of the agents and start the process. Adopting the sequeential task execution.

In [29]:
import time
from crewai import Crew, Process

# forming the crew with predefined agents and tasks
crew = Crew(
    agents = [researcher, blog_writer, image_generator, editor],
    tasks = [research_task, write_task, image_task, edit_task],
    process = Process.sequential
)

# starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': topic})

print(result)



> Entering new CrewAgentExecutor chain...
Thought: To conduct comprehensive research on the impact of AI in modern human society, I need to start by searching for relevant and recent news articles, blogs, and research papers that highlight key insights, trends, and data points.

Action: Search the internet
Action Input: {"search_query": "impact of AI in modern human society"}
 


Search results: Title: The impact of artificial intelligence on human society and bioethics
Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7605294/
Snippet: Studies have shown that AI can reflect the very prejudices humans have tried to overcome. As AI becomes “truly ubiquitous,” it has a tremendous ...
---
Title: What Is The Impact Of Artificial Intelligence (AI) On Society?
Link: https://bernardmarr.com/what-is-the-impact-of-artificial-intelligence-ai-on-society/
Snippet: Artificial intelligence can dramatically improve the efficiencies of our workplaces and can augment the work humans can do. When AI 

## Displaying Results as Markdown

The blog is displayed below along with a DALL-E generated image.

In [32]:
from IPython.display import Markdown

# Display the Markdown content
Markdown("final_blog.md")

**The Impact of AI in Modern Human Society**
=====================================================

<p align="center">
  <img src="https://oaidalleapiprodscus.blob.core.windows.net/private/org-1S6yJfTUME6AFF6HTQsDM3Mi/user-yuvgJkpSpykOGRDLfI7oFkwp/img-erFe3oU3ewjueIdYQuo3Lj8H.png?st=2024-08-09T16%3A58%3A22Z&se=2024-08-09T18%3A58%3A22Z&sp=r&sv=2023-11-03&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-08-09T06%3A44%3A40Z&ske=2024-08-10T06%3A44%3A40Z&sks=b&skv=2023-11-03&sig=2Dh6Za2DOOtPgXRnzN/edY3m3Ftx4YYe97/ENaSH3Bc%3D" alt="The Impact of AI in Modern Human Society" width="500" height="300">
</p>

The integration of Artificial Intelligence (AI) in modern human society has been a topic of discussion for several years. While some people believe that AI will revolutionize the way we live and work, others are concerned about its potential negative impacts. In this article, we will explore the impact of AI in modern human society, highlighting both the benefits and drawbacks of this technology.

**Improving Healthcare and Education**
------------------------------------

One of the most significant benefits of AI is its potential to improve healthcare and education. AI-powered algorithms can analyze vast amounts of data to identify patterns and make predictions, which can help doctors diagnose diseases more accurately and develop personalized treatment plans. Additionally, AI-powered chatbots can help students learn more effectively by providing personalized feedback and guidance. For instance, AI-powered virtual teaching assistants can help students with their homework and provide real-time feedback on their performance.

**Job Displacement and Social Inequality**
-----------------------------------------

However, the increasing use of AI in various industries has also raised concerns about job displacement and social inequality. As AI-powered machines and algorithms take over routine and repetitive tasks, many workers may lose their jobs. This could exacerbate social inequality, as those who have the skills and resources to adapt to the changing job market may thrive, while those who do not may struggle to make ends meet. Furthermore, the increasing use of AI in decision-making processes may also perpetuate existing biases and discrimination, leading to further social inequality.

**Conclusion**
--------------

In conclusion, the impact of AI in modern human society is complex and multifaceted. While AI has the potential to improve healthcare and education, it also raises concerns about job displacement and social inequality. As we move forward, it is essential to develop policies and regulations that ensure the benefits of AI are shared by all, while minimizing its negative impacts. This requires a collaborative effort from governments, industries, and civil society to ensure that AI is developed and used in ways that promote human well-being and dignity.